In [ ]:
from piepy.behavioural_paradigms.detection.wheelDetectionSession import WheelDetectionSession

### 1. Analyse the session

In [ ]:
exp_name = '240325_KC149_detect__no_cam_KC'
w = WheelDetectionSession(exp_name,
                          load_flag=True,
                          skip_google=True)

### 2. Plot the psychometric curve for the session

In [ ]:
from piepy.plotters.basePlotters import *
from piepy.plotters.detection.wheelDetectionSessionPlotter import *
from piepy.plotters.detection.wheelDetectionTrialPlotter import *

In [ ]:
p_psych = DetectionPsychometricPlotter(data=w.data.data,style='print')
p_psych.plot(figsize=(6,4),
       jitter=0.1,
       xaxis_type='linear_spaced',
       doP=True)
# p.save(w.paths.save,w.meta.baredate,w.meta.animalid)
# w.data_paths.analysisPath,w.meta.baredate,w.meta.animalid

### 2. Plot the hit rate performance through the session

In [ ]:
p_perf = DetectionPerformancePlotter(data = w.data.data)
p_perf.plot(seperate_by='stimkey',
            running_window=10,
            figsize=(15,9))

### 2. Plot response times in the session

In [ ]:
p_resp = DetectionResponseTimeScatterCloudPlotter(data = w.data.data)
p_resp.plot_scatter(reaction_of='rig',
                    xaxis_type='linear_spaced')

### 2. Plot number of trial outcomes(hit, miss, early)

In [ ]:
p_outcome = DetectionResponseTypeBarPlotter(w.data.data)
p_outcome.plot()